<h1 align="center">Permutations</h1>

<h2>Changing to correct directory</h2>

In [1]:
import os
from pathlib import Path

os.chdir(Path.cwd().parent)
assert Path.cwd().name == "src" # make sure it is the correct folder

In [2]:
import random
import pandas as pd

import tournament_simulations.data_structures as ds
import tournament_simulations.permutations as pmt
import tournament_simulations.schedules.round_robin as rr

In [3]:
from example_notebooks.matches_df_example import matches_df_permutations
matches = ds.Matches(matches_df_permutations)

matches.df

home   away winner
id date number                     
1  0                a      d      h
   0                b      c      d
   0                a      b      d
   1                c      b      a
   1                d      b      d
   2                c      a      a
   2                a      d      a
   3                b      c      h
2  0              one  three      d
   0              two    one      d
   1              two  three      a
   1              one    two      h
   2            three    two      h
   2            three    one      a

In [4]:
matches.team_names_per_id

id
1         [a, b, c, d]
2    [one, three, two]
Name: teams, dtype: object

<h2>Tournament Scheduler</h2>

> Create a tournament schedules for each "id"

<h3>Same Function for all Tournaments</h3>

In [5]:
random.seed(1)

def create_single_round_robin(num_teams):
    return list(rr.SingleRoundRobin.from_num_teams(num_teams).get_full_schedule(1))

id_to_num_teams = matches.team_names_per_id.apply(len)

scheduler = pmt.TournamentScheduler(
    func_schedule=create_single_round_robin, 
    id_to_parameters=id_to_num_teams.apply(lambda cell: [cell]) # it needs to be a list of parameters
)
scheduler.generate_schedule().series.explode()

id
1    ((1, 2), (0, 3))
1    ((2, 3), (0, 1))
1    ((2, 0), (3, 1))
2           ((0, 1),)
2           ((0, 2),)
2           ((1, 2),)
Name: schedule, dtype: object

<h3>Specific Function for each Tournament</h3>

In [6]:
random.seed(1)

def create_single_round_robin(team_names, num_schedules):
    return list(rr.SingleRoundRobin.from_team_names(team_names).get_full_schedule(num_schedules))

id_to_scheduling_function = pd.Series( # Can be any Mapping entity
    index=matches.df.index.get_level_values("id").unique().tolist(),
    data=[
        lambda team_names: create_single_round_robin(team_names, 3),
        lambda team_names: create_single_round_robin(team_names, 2),
    ]
)

scheduler = pmt.TournamentScheduler(
    func_schedule=id_to_scheduling_function,
    id_to_parameters=matches.team_names_per_id.apply(lambda cell: [cell]) # it needs to be a list of parameters
)
scheduler.generate_schedule().series.explode()

id
1    ((b, c), (a, d))
1    ((c, d), (a, b))
1    ((c, a), (d, b))
1    ((c, d), (a, b))
1    ((d, a), (b, c))
1    ((d, b), (c, a))
1    ((d, b), (c, a))
1    ((d, a), (c, b))
1    ((a, b), (d, c))
2       ((two, one),)
2     ((three, one),)
2     ((three, two),)
2     ((two, three),)
2       ((one, two),)
2     ((three, one),)
Name: schedule, dtype: object

<h2>MatchesPermutations</h2>

> Permute matches in all tournaments

In [7]:
random.seed(1)

def create_double_rr(team_names, num_schedules):
    return list(rr.DoubleRoundRobin.from_team_names(team_names).get_full_schedule(num_schedules))

num_schedule_per_id = matches.home_vs_away_count_per_id.groupby("id", observed=True).max()
id_to_parameters = pd.concat([matches.team_names_per_id, num_schedule_per_id], axis="columns")

scheduler = pmt.TournamentScheduler(
    func_schedule=create_double_rr,
    id_to_parameters=id_to_parameters.agg(tuple, axis="columns"),
)

In [8]:
matches_to_permute = pmt.MatchesPermutations(matches, scheduler)
matches_to_permute.create_n_permutations(2).df

home   away winner
id  date number                     
1@0 0                d      b      d
    0                b      c      d
    0                c      a      a
    1                a      d      h
    1                c      b      a
    2                a      d      a
    2                b      c      h
    3                a      b      d
1@1 0                a      d      a
    0                c      b      a
    0                a      b      d
    1                b      c      h
    1                d      b      d
    2                c      a      a
    2                a      d      h
    3                b      c      d
2@0 0            three    one      a
    0              two    one      d
    1            three    two      h
    1              one  three      d
    2              one    two      h
    2              two  three      a
2@1 0              two  three      a
    0              one  three      d
    1              two    one      d
    1            three    two      h
    2            three    one      a
    2              one    two      h